# #312 Composite Tub
<i>Double composite girders</i>
***

In [ ]:
'''Inputs - Units are N,mm'''
mesh_size = 1000

transverse_spacing = 4000 # Spacing of the two main girders
deck_width = 7000         # Overall width of the concrete deck slab
deck_thk = 200            # Thickness of the concrete deck slab 
shear_stud_stiff = 1000   # Stiffness of the connection between girders and deck

do_solve = False          # Carry out the analysis once the model is constructed

# Loads
surf_load_intensity = -2.5
settlement = -0.01 # m
shrinkage_strain = -0.369764E-3 
construction_load = 10.0 # Construction load (N/mm) used in staged buckling analysis applied to the top flanges

stiff_depth = 200

#### Connect to LUSAS Modeller and create a new model

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
import pandas as pd
import numpy as np
from LPI import *
lusas = get_lusas_modeller()
# if lusas.existsDatabase():
#     raise Exception("This script will create a new model. Please save and close the current model and try again")

# Initialise the helpers module with a reference to LUSAS
from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

# Create a new model
lusas.newProject("Structural", "Composite_Tub.mdl")
# Get a reference to the current model database for convenience
db = lusas.database() 
# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Units N,mm
db.setModelUnits("N,mm,t,s,C")
# Consider only 5 degrees of freedom for thick shell elements
# This prevents diagonal decay of elements connected to the deck with joint elements in 3DOF
db.getOptions().setBoolean("Option 278", False, False, "")

Create a girder class to hold the section dimensions for each segment</br>
We'll create points in the LUSAS model for each section and then use the points to create surfaces joining them.

In [ ]:
class TubGirderSection:
    def __init__(self, tfb:float, tfthk:float, dw:float, tw:float, weboffset:float, bfb:float, bfthk:float, support:str='', bottom_slab_thk:float=0.0):
        self.top_flange_breadth = tfb
        self.top_flange_thk = tfthk
        self.web_depth = dw
        self.web_thk = tw
        self.web_offset = weboffset
        self.no_web_stiffeners = 2
        self.bottom_flange_breadth = bfb
        self.bottom_flange_thk = bfthk
        self.no_bottom_flange_stiffeners = 2
        self.support = support
        self.bottom_slab_thk = bottom_slab_thk
        # We'll save the points created in the model in these lists so we can later use them to define surfaces
        self.bf_points : list[IFPoint] = []
        self.tf1_points : list[IFPoint] = []
        self.tf2_points : list[IFPoint] = []

    def get_bf_coords(self) ->list[float]:
        """Get the coordinates of the bottom flange points"""
        coords = [self.bottom_flange_breadth/2]
        if self.no_bottom_flange_stiffeners == 1:
            coords += [0]
        elif self.no_bottom_flange_stiffeners == 2:
            coords += [self.bottom_flange_breadth/6, -self.bottom_flange_breadth/6]
        coords += [-self.bottom_flange_breadth/2]
        return coords

Read the definitions from the excel sheet

In [ ]:
df = pd.read_excel('312 Composite Tub.xlsx')

Create a list of section definitions, we'll need one per segment + 1 for the end.

In [ ]:
sections : list[TubGirderSection] = []

for i, row in df.iterrows():
    # Create a girder section from the row
    sections.append(TubGirderSection(row['tfb'], row['tfthk'], row['dw'], row['tw'], row['webOffset'], row['bfb'], row['bfthk'], row['support'], 0.0))

# Get the lengths of the segments from the DataFrame
segments_lengths = list(df['SegLength'][:-1]) # Exclude the last segment length as it is not used


Create a list of stiffener thicknesses that will be positioned at each change in section. A zero thickness indicates no stiffener.

In [ ]:
stiffener_thicknesses = list(df['StiffThk'])

In [ ]:
# When calling the LPI from an external process as we are here a speedup can be gained in v22 and later by disabling the UI
if lusas.getMajorVersionNumber() >=22:
    lusas.enableUI(False)

Create model attributes

In [ ]:
# Create the mesh attribute
shell_mesh_attr = db.createMeshSurface(f"Shell Mesh {mesh_size}mm").setRegularSize("QTS4", mesh_size, True)

# Bar mesh attributes
bracing_mesh_attr = db.createMeshLine("Bar Mesh").setNumber("BRS2", 1)

# Stiffener mesh attribute
stiffener_mesh_attr = db.createMeshLine("Stiffener Mesh").setSize("BMI21", mesh_size)

# Steel Material attribute
steel_material = db.createIsotropicMaterial("Steel", 200_000, 0.3, 7.8e-9, 10e-6)

# Concrete Material attribute
concrete_material           = db.createIsotropicMaterial("Concrete", 30_000, 0.2, 2.4e-9, 10e-6)
concrete_material_long_term = db.createIsotropicMaterial("Concrete (Long Term)", 30_000/3, 0.2, 2.4e-9, 10e-6)
concrete_material_cracked   = db.createOrthotropicThickMaterial("Concrete Cracked", 15_000, 30_000, 10_000, 10_000, 10_000, 0.3, 0.0, 2.4E-9, 0.0)
concrete_material_cracked.setValue("ax", 10e-6, 0).setValue("ay", 10e-6, 0)

# Bracing geometic attribute
bracing_section_attr = db.createGeometricLine("Bracing Section")
bracing_section_attr.setFromLibrary("UK Sections", "Equal Angles (Advance)", "150x150x12 UKEA", 0, 0, 0)

# Surface attributes will be created for each surface thickness below

# Connection between girders and deck is made with stiff joint elements.
# Create a joint mesh and material attribute for this connection
shear_stud_mesh_attr = db.createMeshLine("Shear Connection Mesh").setSize("JNT4", mesh_size)
shear_stud_material_attr = db.createSpringJointMaterial("Shear Connection Material", [shear_stud_stiff,shear_stud_stiff,shear_stud_stiff])
shear_stud_material_attr.setValue("Assignment", "Line")

# Support attributes
pinned = db.createSupportStructural("Fixed in XZ").setStructural("R", "F", "R", "F", "F", "F", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in Z").setStructural("F", "F", "R", "F", "F", "F", "F", "F", "C", "F")

In [ ]:
# Deactivate attribute disables elements to which its assigned
deactive_attr = db.createDeactivate("Deactivate").setDeactivate("activeMesh", 100.0, 1.0E-6)
# Activate attribute reactivates attributes in the loadcase to which it's assigned
activate_attr = db.createActivate("Activate")
# Local coords for orthotropic material assignment
local_coord_attr = db.createLocalCartesianXYAttr("LCrd1", 0.0, [0,0,0]).setAxesType("Cartesian")

In [ ]:
# Construction load attribute
construction_load_attr = db.createLoadingGlobalDistributed("Global Distributed Load")
construction_load_attr.setGlobalDistributed("Length", 0.0, 0.0, -construction_load, 0.0, 0.0, 0.0, 0.0, 0.0, False, 0.0)


Create all the points for each section, saving the returned point in the Girder section definition. Use the helper fucntions library for create_point function

In [ ]:

x = 0
for i in range(0, len(sections)):
    # Determine the longitudinal position (x coord) for each section
    if i > 0 : x += segments_lengths[i-1]
    # Determine the section to use at each longitudinal position
    s = sections[i]
    # Create the bottom flange points
    for y in s.get_bf_coords():
        s.bf_points.append(Helpers.create_point(x, y, -s.web_depth))

    # Create the top flange points
    y = s.bottom_flange_breadth /2 + s.web_offset
    s.tf1_points.append(Helpers.create_point(x, y +s.top_flange_breadth/2, 0.0))
    s.tf1_points.append(Helpers.create_point(x, y,                         0.0))
    s.tf1_points.append(Helpers.create_point(x, y -s.top_flange_breadth/2, 0.0))
    s.tf2_points.append(Helpers.create_point(x, -y +s.top_flange_breadth/2, 0.0))
    s.tf2_points.append(Helpers.create_point(x, -y,                         0.0))
    s.tf2_points.append(Helpers.create_point(x, -y -s.top_flange_breadth/2, 0.0))

Now join up the segment points. First create helper function to create surfaces and assign a geometric thickness attribute

In [ ]:
def create_surface(pnts: list, thk:float, ecc:float, group:str) -> IFSurface:

    geometry_data = lusas.geometryData().setAllDefaults()
    geometry_data.setLowerOrderGeometryType("points")

    obs = lusas.newObjectSet().add(pnts)

    surface = obs.createSurface(geometry_data).getObject("Surface")
    db.getGroupByName(group).add(surface, "Surfaces")

    name = f"{group} ({thk:.1f}mm)"

    db.createGeometricSurface(name).setSurface(thk, ecc).assignTo(surface)
    return surface

Create groups to contain the girder surfaces

In [ ]:
db.createEmptyGroup("Top Flange")
db.createEmptyGroup("Web")
db.createEmptyGroup("Bottom Flange")
db.createEmptyGroup("Web Stiffeners")
db.createEmptyGroup("Deck")
db.createEmptyGroup("Bottom Slab")

Here we'll create the surfaces between each section definition using the points we created earlier

In [ ]:
for i in range(0, len(segments_lengths)):

    s1:TubGirderSection = sections[i]   # Section definition at the start of the segment
    s2:TubGirderSection = sections[i+1] # Section definition at the end of the segment

    for j in range(0, len(s1.bf_points)-1):
        create_surface([ s1.bf_points[j], s2.bf_points[j], s2.bf_points[j+1], s1.bf_points[j+1] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")



    #create_surface([ s1.bf_points[0], s2.bf_points[0], s2.bf_points[-1], s1.bf_points[-1] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")

    create_surface([ s1.bf_points[0], s2.bf_points[0], s2.tf1_points[1], s1.tf1_points[1] ], s1.web_thk,           0.0, "Web")
    create_surface([ s1.bf_points[-1], s2.bf_points[-1], s2.tf2_points[1], s1.tf2_points[1] ], s1.web_thk,           0.0, "Web")

    create_surface([ s1.tf1_points[1], s2.tf1_points[1], s2.tf1_points[0], s1.tf1_points[0] ], s1.top_flange_thk,    0.0, "Top Flange")
    create_surface([ s1.tf1_points[2], s2.tf1_points[2], s2.tf1_points[1], s1.tf1_points[1] ], s1.top_flange_thk,    0.0, "Top Flange")

    create_surface([ s1.tf2_points[1], s2.tf2_points[1], s2.tf2_points[0], s1.tf2_points[0] ], s1.top_flange_thk,    0.0, "Top Flange")
    create_surface([ s1.tf2_points[2], s2.tf2_points[2], s2.tf2_points[1], s1.tf2_points[1] ], s1.top_flange_thk,    0.0, "Top Flange")


Create stiffeners surfaces

In [ ]:
for i, thk in enumerate(stiffener_thicknesses):

    if thk > 0:

        s = sections[i]   # Section definition at the start of the segment

        line1 = Helpers.get_line_between_points(s.tf1_points[1], s.bf_points[0])
        line2 = Helpers.get_line_between_points(s.tf2_points[1], s.bf_points[-1])

        bflines = []
        for j in range(len(s.get_bf_coords())-1):
            bflines.append( Helpers.get_line_between_points(s.bf_points[j], s.bf_points[j+1]) )

        name = f"Stiffener ({stiffener_thicknesses[i]:.1f}mm)"
        stiffener_section_attr = Helpers.create_rectangular_section(db, name, stiffener_thicknesses[i], stiff_depth)
        stiffener_section_attr.setValue("Ez", -stiff_depth/2)
        
        stiffener_section_attr.assignTo(line1)
        stiffener_section_attr.assignTo(line2)
        stiffener_section_attr.assignTo(bflines)
        stiffener_mesh_attr.assignTo(line1)
        stiffener_mesh_attr.assignTo(line2)
        stiffener_mesh_attr.assignTo(bflines)
        steel_material.assignTo(line1)
        steel_material.assignTo(line2)  
        steel_material.assignTo(bflines)


Assign a shell mesh attribute and steel material attribute to all surfaces

In [ ]:
# Assign mesh and steel material to all girder surfaces
assignment = lusas.assignment().setAllDefaults()
shell_mesh_attr.assignTo("Surfaces", assignment)
steel_material.assignTo("Surfaces", assignment)

Support attributes

In [ ]:
# To assign these support attributes we'll need to get hold of the lines. We can do this via the points that have created for each section

# Assign supports to the end and central section
for section in sections:

    if not isinstance(section.support, str) or section.support == '': continue # Skip sections that are not supported

    bflines = []
    for i in range(len(section.get_bf_coords())-1):
        bflines.append( Helpers.get_line_between_points(section.bf_points[i], section.bf_points[i+1]) )

    if section.support == 'slide':
        slide.assignTo(bflines)
    elif section.support == 'pinned':
        pinned.assignTo(bflines)
    else:
        print("Unrecognised support type:", section.support)

In [ ]:
# Set Default Attributes for bracing members
db.setAsDefault("Mesh", bracing_mesh_attr)
db.setAsDefault("Material", steel_material)
db.setAsDefault("Line Geometric", bracing_section_attr)

In [ ]:
# Create the bracing members
for i in range(0, len(segments_lengths)+1):
    if stiffener_thicknesses[i] > 0:
        line1 = Helpers.create_line_from_points(sections[i].bf_points[0], sections[i].tf2_points[1])
        line2 = Helpers.create_line_from_points(sections[i].tf1_points[1], sections[i].bf_points[-1])
        # line3 = Helpers.create_line_from_points(sections[i].bf_points[1], sections[i].bf_points[1])
        line4 = Helpers.create_line_from_points(sections[i].tf1_points[1], sections[i].tf2_points[1])

In [ ]:
# Plan bracing
start_segment = 1
for ix, i in enumerate(range(start_segment, len(segments_lengths) - start_segment)):
    if ix % 2 == 0:
        line = Helpers.create_line_from_points(sections[i].tf1_points[1], sections[i+1].tf2_points[1])
    else:
        line = Helpers.create_line_from_points(sections[i].tf2_points[1], sections[i+1].tf1_points[1])


In [ ]:
# Clear default attribute assignments
db.setAsDefault("Mesh", "None")
db.setAsDefault("Material", "None")
db.setAsDefault("Line Geometric", "None")

# Slab Deck

In [ ]:
x_grid = [0]
for s in segments_lengths:
    x_grid.append(x_grid[-1]+s)

#y = s.bottom_flange_breadth /2 + s.web_offset
y_grid = [-deck_width/2, -transverse_spacing/2, 0.0, transverse_spacing/2, deck_width/2]

In [ ]:
# Create the actual points in the model
deck_points = [[Helpers.create_point(x, y, deck_thk/2) for x in x_grid] for y in y_grid]
# [Y][X]

# Create the surfaces for the deck
for iy in range(0, len(y_grid)-1):
    for ix in range(0, len(x_grid)-1):
        create_surface([ deck_points[iy][ix], deck_points[iy][ix+1], deck_points[iy+1][ix+1], deck_points[iy+1][ix] ], deck_thk, 0.0, "Deck")

In [ ]:
shell_mesh_attr.assignTo("Group", "Deck")
concrete_material.assignTo("Group", "Deck")

# Deck to Girder connection

In [ ]:
# Helper function to find the lines in the top flanges and deck surfaces
def get_lines_x(points:list[IFPoint])->list[IFLine]:
    lines = []
    for point in points:
        for hof in point.getHOFs():
            if hof.getTypeCode() == 2 : # Line
                # Check it is aligned with and parallel to the x axis
                if abs(hof.getEndPosition()[1] - hof.getStartPosition()[1]) < 1e-3 and \
                   abs(hof.getEndPosition()[2] - hof.getStartPosition()[2]) < 1e-3:
                    # Add to the list if not already in there
                    if not hof in lines:
                        lines.append(hof)
    return lines

# List of lines in section 1 top flange
lines_s1_tf   = get_lines_x([s.tf2_points[1] for s in sections])
# List of lines in deck corresponding to the lines in section 1
lines_s1_deck = get_lines_x([p for p in deck_points[1]])
# List of lines in section 1 top flange
lines_s2_tf   = get_lines_x([s.tf1_points[1] for s in sections])
# List of lines in deck corresponding to the lines in section 2
lines_s2_deck = get_lines_x([p for p in deck_points[-2]])
# Default assignment
assignment = lusas.assignment().setAllDefaults()
# Assign the mesh between corresponding lines of the girder top flange and deck
for i in range(0, len(lines_s1_tf)):
    objs = lusas.newObjectSet().add(lines_s1_deck[i]).add(lines_s1_tf[i])
    shear_stud_mesh_attr.assignTo(objs,assignment)
    del objs
# Assign the mesh between corresponding lines of the girder top flange and deck
for i in range(0, len(lines_s2_tf)):
    objs = lusas.newObjectSet().add(lines_s2_deck[i]).add(lines_s2_tf[i])
    shear_stud_mesh_attr.assignTo(objs,assignment)
    del objs
# Assign the material for the joint elements to the primary line of the joint assignment
shear_stud_material_attr.assignTo(lines_s1_deck, assignment)
shear_stud_material_attr.assignTo(lines_s2_deck, assignment)

## Set loadcase and view

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
# The loadcase will be used only to test the model is working correctly
# NOTE: getLoadset and setName function returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase").addGravity(True)
db.getAnalysis("Analysis 1").setName("00 Base")

Set the model to display the assigned plate thicknesses and supports in an isometric orientation

In [ ]:
lusas.view().insertGeometryLayer() # Make sure the drawing layers exist before accessing them
lusas.view().insertAttributesLayer()

lusas.view().geometry().autoColourByAttributes("Geometric", True)
lusas.view().attributes().visualiseAll("Supports")
lusas.view().setIsometric()

In [ ]:
# Mesh the model
db.resetMesh()
db.updateMesh()

In [ ]:
# Solve all the analyses
if do_solve:
    for analysis in db.getAnalyses():
        analysis.solve(True)
    db.openAllResults(False)

In [ ]:
# Re-enable the UI. Note if the script failed prior to this, this cell must be run manually to re-enable the UI
lusas.enableUI(True)